In [ ]:
import transformers
print(f"현재 transformers 버전: {transformers.__version__}")
import sys
import torch
import types
from unittest.mock import MagicMock
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import os

# ==========================================
# [필살기] Flash Attention 완벽하게 속이기
# ==========================================
if "flash_attn" in sys.modules:
    del sys.modules["flash_attn"]

mock_flash_attn = types.ModuleType("flash_attn")
mock_flash_attn.__spec__ = types.SimpleNamespace(
    name="flash_attn", 
    loader=None, 
    origin="mock", 
    submodule_search_locations=None
)
mock_flash_attn.__file__ = "mock_flash_attn.py"
mock_flash_attn.bert_padding = MagicMock()
mock_flash_attn.flash_attn_interface = MagicMock()
mock_flash_attn.flash_attn_func = MagicMock()

sys.modules["flash_attn"] = mock_flash_attn
sys.modules["flash_attn.bert_padding"] = mock_flash_attn.bert_padding
sys.modules["flash_attn.flash_attn_interface"] = mock_flash_attn.flash_attn_interface

print("✅ Flash Attention 가짜 모듈 등록 완료")

# ==========================================
# 모델 로드 (VARCO-VISION-2.0-1.7B)
# ==========================================
device = "cuda" if torch.cuda.is_available() else "cpu"
# RTX 3060 6GB 메모리 확보를 위해 반드시 float16 사용
dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "ncsoft/VARCO-VISION-2.0-1.7B"

print(f"{model_id} 로드 시작...")

# 모델 로드 (메모리 최적화 옵션 포함)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=dtype,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to(device)

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

print(f"🎉 VARCO 모델 로드 성공! 수고했어 덩우야!")

# ==========================================
# OCR 실행 함수
# ==========================================
def run_varco_ocr(image_path):
    """
    이미지 경로를 받아 VARCO-VISION 모델로 텍스트를 추출합니다.
    """
    if not os.path.exists(image_path):
        return f"에러: '{image_path}' 파일을 찾을 수 없습니다."

    # 이미지 열기
    image = Image.open(image_path).convert("RGB")

    # VARCO 전용 프롬프트 구성
    # 그림일기 텍스트 추출에 최적화된 질문
    prompt = "이미지에 적힌 글자를 빠짐없이 텍스트로 변환해줘."
    
    # 입력 데이터 전처리
    inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, dtype)

    # 생성 (Inference)
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=1024,
            do_sample=False,
            temperature=0.0,
            eos_token_id=processor.tokenizer.eos_token_id
        )

    # 결과 디코딩
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return generated_text

# ==========================================
# 실행부
# ==========================================
# 전처리가 완료된 파일명을 입력하세요
image_file_path = "cleaned_text_box.png" 

try:
    print("-" * 30)
    print(f"📄 파일명: {image_file_path}")
    print("⏳ OCR 분석 중...")
    
    result_text = run_varco_ocr(image_file_path)
    
    print("✨ VARCO OCR 결과:")
    print(result_text)
    print("-" * 30)
except Exception as e:
    print(f"❌ 실행 중 에러 발생: {e}")